In [ ]:
import os
import numpy as np
import pandas as pd
from itertools import product

import matplotlib.pyplot as plt 



from scipy.signal import hilbert
from scipy.signal import butter, lfilter

from skimage.morphology import square, dilation, disk
from skimage.feature import canny
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,denoise_wavelet, estimate_sigma)
from skimage.filters import median
from skimage.io import imread, imsave
from skimage.color import rgb2gray
import cv2


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, merge, Reshape, BatchNormalization, LeakyReLU,  Concatenate, Activation, Add, Conv2DTranspose, concatenate
from keras import optimizers
from keras import activations
from keras.callbacks import ModelCheckpoint
from keras import backend as K

#from Networks import *
#from Comp2_func import *
#from IOU import *

#from datetime import datetime
import gc
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb

In [ ]:
def im_load(size):
    img_path= 'C:/Users/eitanp/Desktop/ML/Competition/Salt_Win/train_data/train_images/'
    msk_path= 'C:/Users/eitanp/Desktop/ML/Competition/Salt_Win/train_data/train_masks/'
    imx_bank= []
    imy_bank= []
    depths= []
    depth_data= pd.read_csv('depths.csv')
    for root, dirs, files in os.walk(img_path + '.'):  
        for file_name in files:
            imx= imread(img_path + file_name)
            imy= imread(msk_path + file_name)
            depth= depth_data['z'][depth_data['id']==file_name.split('.')[0]]

            imx= rgb2gray(imx)
            imy= imy / 65535
            if imx.max()>0:
                
                #imx= (imx - np.mean(imx)) / np.std(imx)
                #size= (63, 63)
                imx= cv2.resize(imx, size)
                imy= cv2.resize(imy, size)
                
                imx_bank.append(imx)
                imy_bank.append(imy)
                depths.append(depth.values * np.ones((size[0] * size[1], )).astype(int))  

    imx_bank= np.array(imx_bank)
    imy_bank= np.array(imy_bank)
    depths= np.array(depths)
    return imx_bank, imy_bank, depths

def test_load(size):
    img_path= 'C:/Users/eitanp/Desktop/ML/Competition/Salt_Win/test_images/'
    imx_bank= []
    names_bank= []
    names0_bank= []
    depths= []
    depth_data= pd.read_csv('depths.csv')
    for root, dirs, files in os.walk(img_path + '.'):  
        for file_name in files:
            imx= imread(img_path + file_name)
           
            imx= rgb2gray(imx)
            #print(imx.shape)
            if imx.max()>0:
                imx= cv2.resize(imx, size)
                #print(imx.shape)

                depth= depth_data['z'][depth_data['id']==file_name.split('.')[0]]

                names_bank.append(file_name.split('.')[0])

                imx_bank.append(imx)
                depths.append(depth.values * np.ones((size[0] * size[1], )).astype(int)) 
            else:
                names0_bank.append(file_name.split('.')[0])
    
    imx_bank= np.array(imx_bank)
    #print(imx_bank.shape)
    
    depths= np.array(depths)
    return imx_bank, depths, names_bank, names0_bank

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def BP_filter(data, lowcut, highcut, fs, order, axis):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    if axis=='vertical':
        data= np.rot90(data)
    y = lfilter(b, a, data)
    if axis== 'vertical':
        y= np.rot90(y, k=1, axes=(1,0))
    return y

def BP_filter90(data, lowcut, highcut, fs, order):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    #data= np.rot90(data)
    y = lfilter(b, a, data)
    #y= np.rot90(y, k=1, axes=(1,0))
    return y

def bp(X, lowcut, highcut, order, axis):
    imx_bank= []
    for imx in X:
        bp= BP_filter(imx, lowcut=lowcut, highcut=highcut, fs=1/0.004, order=order, axis=axis)
        imx_bank.append(bp/1.3)
    imx_bank= np.array(imx_bank)
    return imx_bank


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def binary_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + (1 - dice_coef(y_true, y_pred))

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef1(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = np.sum(np.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (np.sum(np.square(y_true),-1) + np.sum(np.square(y_pred),-1) + smooth)


# Show images
def show_im2(X, y):
    #imy= np.squeeze(y)
    #imx_show= np.squeeze(X)
    #imp= np.squeeze(imp, axis=[0, -1])

    fig, (ax1, ax2)= plt.subplots(1, 2)
    ax1.imshow(X)
    ax2.imshow(y)
    plt.show()


def show_im3(X, y, imp):
    #imy= np.squeeze(y)
    #imx_show= np.squeeze(X)
    #imp= np.squeeze(imp, axis=[0, -1])

    fig, (ax1, ax2, ax3)= plt.subplots(1, 3)
    ax1.imshow(X)
    ax2.imshow(y)
    ax3.imshow(imp)
    plt.show()
    
def show_im4(X, y, imp, imp1):
    #imy= np.squeeze(y)
    #imx_show= np.squeeze(X)
    #imp= np.squeeze(imp, axis=[0, -1])

    fig, (ax1, ax2, ax3, ax4)= plt.subplots(1, 4)
    ax1.imshow(X)
    ax2.imshow(y)
    ax3.imshow(imp)
    ax4.imshow(imp1)
    
    plt.show()
    
    
# Results
def rle (img):
    result=''
    flat_img = img.flatten(order='F')
    flat_img = np.where(flat_img > 0.5, 1, 0).astype(np.uint8)

    starts = (flat_img == 1) & (np.insert(flat_img[:-1], 0, 0)==0)
    ends = (flat_img == 1) & (np.insert(flat_img[1:], len(flat_img) - 1, 0)==0)
    starts_ix = np.where(starts)[0]
    ends_ix = np.where(ends)[0]
    lengths = (ends_ix - starts_ix) + 1
    for s, l in zip(starts_ix + 1, lengths):
        result+= '{} {} '.format(s, l)
    return result


In [ ]:
mid_path= 'Mid_data/'
models_path= 'Models/'

# Load Raw Data

In [ ]:
size= (96, 96)
size= (96, 96)
print('Loading train images')
X, y, d= im_load(size)
X= X.astype(np.float32)
print(X.shape)

print('Loading test images')
T, dt, names, names0= test_load(size)
T= T.astype(np.float32)
print(T.shape)
#np.save(mid_path + 'X' + str(size[0]), X); np.save(mid_path + 'T' + str(size[0]), T)
#np.save(mid_path + 'y' + str(size[0]), y)
#np.save(mid_path + 'd_f' + str(size[0]), d.flatten())
#np.save(mid_path + 'dt_f' + str(size[0]), dt.flatten())
np.save(mid_path + 'names', names); np.save(mid_path + 'names0', names0)

### Normalizing data for QC

In [ ]:
x= X.reshape(-1, 96 * 96); t= T.reshape(-1, 96 * 96)
Xn= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
Tn= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)

### QC

In [ ]:
n=5
lim_range= (-2, 2)
fig, (ax1, ax2)= plt.subplots(1, 2)
ax1.hist(X[n].flatten(), 100)
ax1.set_xlim(lim_range)
ax2.hist(Xn[n].flatten(), 100)
ax2.set_xlim(lim_range)
plt.show()

# AutoEncoding

In [ ]:
def AutoD1():
    input_img = Input(shape=(96, 96, 1))
    #lr= 0.5
    #decay= 5e-5
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    #opt= optimizers.Adam(lr = lr, decay= decay)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    return autoencoder

def AutoD2():
    input_img = Input(shape=(96, 96, 1))
    #lr= 0.5
    #decay= 5e-5
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    #opt= optimizers.Adam(lr = lr, decay= decay)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    return autoencoder

### Load data

In [ ]:
X= np.load(mid_path + 'X96.npy'); T= np.load(mid_path + 'T96.npy')
y= np.load(mid_path + 'y96.npy')

In [ ]:
y_in= X
y_in= y_in.reshape((-1, 96 , 96, 1))

### Train 2 layers for 10 epochs

In [ ]:
autoencoderD2E10= AutoD2()
autoencoderD2E10.fit(y_in, y_in,
                epochs=10,
                batch_size=128,
                shuffle=True)
autoencoderD2E10.save(models_path + 'AutoD2E10.hdf5')

### QC

In [ ]:
predD2E10= autoencoderD2E10.predict(y_in)
predD2E10= predD2E10.squeeze(axis= -1)

y_inT= T.reshape((-1, 96 , 96, 1))
predD2E10T= autoencoderD2E10.predict(y_inT)
predD2E10T= predD2E10T.squeeze(axis= -1)

np.save(mid_path + 'predD2E10', predD2E10); np.save(mid_path + 'predD2E10T', predD2E10T)

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im4(X[i], predD2E10[i], X[i] - predD2E10[i], y[i])

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3( T[i], predD2E10T[i], T[i] - predD2E10T[i])

In [ ]:
n=45
fig, (ax1, ax2)= plt.subplots(1, 2)
ax1.hist(X[n].flatten(), 100)
ax1.set_xlim(0,1)
ax2.hist(predD2E10[n].flatten(), 100)
ax2.set_xlim(0,1)
plt.show()

### Train 2 layers for 50 epochs

In [ ]:
autoencoderD2E50= AutoD2()
autoencoderD2E50.fit(y_in, y_in,
                epochs=50,
                batch_size=128,
                shuffle=True)
autoencoderD2E50.save(models_path + 'AutoD2E50.hdf5')

### QC

In [ ]:
predD2E50= autoencoderD2E50.predict(y_in)
predD2E50= predD2E50.squeeze(axis= -1)

y_inT= T.reshape((-1, 96 , 96, 1))
predD2E50T= autoencoderD2E50.predict(y_inT)
predD2E50T= predD2E50T.squeeze(axis= -1)

np.save(mid_path + 'predD2E50', predD2E50); np.save(mid_path + 'predD2E50T', predD2E50T)


In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im4(X[i], predD2E50[i],  X[i] - predD2E50[i], y[i])

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3( T[i], predD2E50T[i], T[i] - predD2E50T[i])

### Train 1 layer for 10 epochs

In [ ]:
autoencoderD1E10= AutoD1()
autoencoderD1E10.fit(y_in, y_in,
                epochs=10,
                batch_size=128,
                shuffle=True)
autoencoderD1E10.save(models_path + 'AutoD1E10.hdf5')

### QC

In [ ]:
predD1E10= autoencoderD1E10.predict(y_in)
predD1E10= predD1E10.squeeze(axis= -1)

predD1E10T= autoencoderD1E10.predict(y_inT)
predD1E10T= predD1E10T.squeeze(axis= -1)

np.save(mid_path + 'predD1E10', predD1E10); np.save(mid_path + 'predD1E10T', predD1E10T)

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im4(X[i], predD1E10[i],  X[i] - predD1E10[i], y[i])

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3(T[i], predD1E10T[i],  T[i] - predD1E10T[i])

In [ ]:
#x= pred1.reshape(-1, 96 * 96); t= pred1T.reshape(-1, 96 * 96)
#Xn= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
#Tn= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)

In [ ]:
n=5
lim_range= (0, 1)
fig, (ax1, ax2, ax3)= plt.subplots(1, 3)
ax1.hist(X[n].flatten(), 100)
ax1.set_xlim(lim_range)
ax2.hist(predD1E10[n].flatten(), 100)
ax2.set_xlim(lim_range)
ax3.hist((X[n] - predD1E10[n]).flatten(), 100)
ax3.set_xlim(-1,1)
plt.show()

In [ ]:
n=5
lim_range= (0, 1)
fig, (ax1, ax2, ax3)= plt.subplots(1, 3)
ax1.hist(T[n].flatten(), 100)
ax1.set_xlim(lim_range)
ax2.hist(predD1E10T[n].flatten(), 100)
ax2.set_xlim(lim_range)
ax3.hist((T[n] - predD1E10T[n]).flatten(), 100)
ax3.set_xlim(-1,1)
plt.show()

# Segmentation

In [ ]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def binary_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + (1 - dice_coef(y_true, y_pred))

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef1(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = np.sum(np.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (np.sum(np.square(y_true),-1) + np.sum(np.square(y_pred),-1) + smooth)

In [ ]:
def conv_block(m, dim, acti, bn, res, do=0):
    n = Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block(m, dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block(n, dim, acti, bn, res)
    else:
        m = conv_block(m, dim, acti, bn, res, do)
    return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
         dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
    i = Input(shape=img_shape)
    o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    o = Conv2D(out_ch, 1)(o)
    o = Activation('sigmoid')(o)
    return Model(inputs=i, outputs=o)

def Logit_UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
         dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
    i = Input(shape=img_shape)
    o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    o = Conv2D(out_ch, 1)(o)
    return Model(inputs=i, outputs=o)

### Load data

In [ ]:
D1E10_comp1= np.load(mid_path + 'predD1E10.npy'); D1E10T_comp1= np.load(mid_path + 'predD1E10T.npy')
D2E50_comp1= np.load(mid_path + 'predD2E50.npy'); D2E50T_comp1= np.load(mid_path + 'predD2E50T.npy')
X= np.load(mid_path + 'X96.npy'); T= np.load(mid_path + 'T96.npy')
y= np.load(mid_path + 'y96.npy')

In [ ]:
yin= np.expand_dims(y, axis= -1)
yin.shape

### Train for predD2E50

In [ ]:
x= D2E50_comp1.reshape(-1, 96 * 96); 
X1= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 

x= (X - D2E50_comp1).reshape(-1, 96 * 96); 
X2= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 

In [ ]:
Xin= X1
Xin= Xin.reshape(-1, 96, 96)
Xin= np.expand_dims(Xin, axis= -1)
Xin.shape

In [ ]:
shape= Xin[0].shape
model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])

model_name= 'D2E50_comp1_seg'
checkpoint1 = ModelCheckpoint(models_path + model_name + '_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=False, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(models_path + model_name + '_weights_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint1, checkpoint2]

epochs= 5
batch_size= 128
history= model.fit(Xin, yin, epochs=epochs,
                   batch_size=batch_size, verbose=1,
                   #validation_data=(X, y),
                   callbacks=callbacks_list)

In [ ]:
Xin= X2
Xin= Xin.reshape(-1, 96, 96)
Xin= np.expand_dims(Xin, axis= -1)
Xin.shape

In [ ]:
shape= Xin[0].shape
model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])

model_name= 'D2E50_comp2_seg'
checkpoint1 = ModelCheckpoint(models_path + model_name + '_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=False, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(models_path + model_name + '_weights_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint1, checkpoint2]

epochs= 5
batch_size= 128
history= model.fit(Xin, yin, epochs=epochs,
                   batch_size=batch_size, verbose=1,
                   #validation_data=(X, y),
                   callbacks=callbacks_list)

### Train for predD1E10

In [ ]:
x= (np.log1p(D1E10_comp1)).reshape(-1, 96 * 96); #t= D1E10T_comp1.reshape(-1, 96 * 96)
X1= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 

x= (np.log1p((X - D1E10_comp1))).reshape(-1, 96 * 96); 
X2= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 

In [ ]:
Xin= X1
Xin= Xin.reshape(-1, 96, 96)
Xin= np.expand_dims(Xin, axis= -1)
Xin.shape

In [ ]:
shape= Xin[0].shape
model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])

model_name= 'D1E10_comp1_seg'
checkpoint1 = ModelCheckpoint(models_path + model_name + '_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=False, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(models_path + model_name + '_weights_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint1, checkpoint2]

epochs= 5
batch_size= 128
history= model.fit(Xin, yin, epochs=epochs,
                   batch_size=batch_size, verbose=1,
                   #validation_data=(X, y),
                   callbacks=callbacks_list)

In [ ]:
Xin= X2
Xin= Xin.reshape(-1, 96, 96)
Xin= np.expand_dims(Xin, axis= -1)
Xin.shape

In [ ]:
shape= Xin[0].shape
model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])

model_name= 'D1E10_comp2_seg'
checkpoint1 = ModelCheckpoint(models_path + model_name + '_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=False, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(models_path + model_name + '_weights_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint1, checkpoint2]

epochs= 5
batch_size= 128
history= model.fit(Xin, yin, epochs=epochs,
                   batch_size=batch_size, verbose=1,
                   #validation_data=(X, y),
                   callbacks=callbacks_list)

### QC

In [ ]:
D1E10_comp1= np.load(mid_path + 'predD1E10.npy'); D1E10T_comp1= np.load(mid_path + 'predD1E10T.npy')
X= np.load(mid_path + 'X96.npy'); T= np.load(mid_path + 'T96.npy')
y= np.load(mid_path + 'y96.npy')

In [ ]:
yin= np.expand_dims(y, axis= -1)
yin.shape

In [ ]:
x= D1E10_comp1.reshape(-1, 96 * 96); t= D1E10T_comp1.reshape(-1, 96 * 96)
X1= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
T1= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)


x= (X - D1E10_comp1).reshape(-1, 96 * 96); t= (T - D1E10T_comp1).reshape(-1, 96 * 96)
X2= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
T2= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)



X1= X1.reshape(-1, 96, 96)
X1= np.expand_dims(X1, axis= -1)
print(X1.shape)

X2= X2.reshape(-1, 96, 96)
X2= np.expand_dims(X2, axis= -1)
print(X2.shape)


T1= T1.reshape(-1, 96, 96)
T1= np.expand_dims(T1, axis= -1)
print(T1.shape)


T2= T2.reshape(-1, 96, 96)
T2= np.expand_dims(T2, axis= -1)
print(T2.shape)

In [ ]:
model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])
model.load_weights(models_path + 'D1E10_comp2_seg_weights_ep05_best.hdf5')

In [ ]:
pX= model.predict(X2[:300])
pT= model.predict(T2[:300])

In [ ]:
pX= np.squeeze(pX, axis=-1)
pT= np.squeeze(pT, axis=-1)

In [ ]:
pX.max(), pT.max()

In [ ]:
level= pX.flatten()
y_f= y[:300].flatten()
dice= []
thresholds= np.arange(0.1, level.max(), 0.01)
for threshold in thresholds:
    p_test= np.zeros((level.shape))

    p_test[level > threshold]= 1
    p_test= p_test.astype(np.float32)
    y_true= y_f.astype(np.float32)

    dice.append(dice_coef1(y_true, p_test.squeeze()))
plt.plot(thresholds, dice)
max_dice= np.array(dice).max()
best_th= thresholds[np.argmax(np.array(dice))]
print('max dice is ' + str(max_dice) + 'at threshold ' + str(best_th))

In [ ]:
p_finalX= np.zeros((pX.shape))
p_finalX[pX > best_th]= 1

p_finalT= np.zeros((pT.shape))
p_finalT[pT > best_th]= 1

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3(X[i], p_finalX[i]  , y[i])

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3(T[i], p_finalT[i]  , p_finalT[i])

### Train for X

In [ ]:
x= X.reshape(-1, 96 * 96)
Xin= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 

In [ ]:
Xin= Xin.reshape(-1, 96, 96)
Xin= np.expand_dims(Xin, axis= -1)
Xin.shape

In [ ]:
shape= Xin[0].shape
model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])

model_name= 'X_seg'
checkpoint1 = ModelCheckpoint(models_path + model_name + '_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=False, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(models_path + model_name + '_weights_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint1, checkpoint2]

epochs= 5
batch_size= 128
history= model.fit(Xin, yin, epochs=epochs,
                   batch_size=batch_size, verbose=1,
                   #validation_data=(X, y),
                   callbacks=callbacks_list)

### Train for Log(1 + X)

In [ ]:
x= (np.log1p(X)).reshape(-1, 96 * 96)
Xin= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 

In [ ]:
Xin= Xin.reshape(-1, 96, 96)
Xin= np.expand_dims(Xin, axis= -1)
Xin.shape

In [ ]:
shape= Xin[0].shape
model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])

model_name= 'log1pX_seg'
checkpoint1 = ModelCheckpoint(models_path + model_name + '_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=False, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(models_path + model_name + '_weights_ep{epoch:02d}_best.hdf5', monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint1, checkpoint2]

epochs= 50
batch_size= 128
history= model.fit(Xin, yin, epochs=epochs,
                   batch_size=batch_size, verbose=1,
                   #validation_data=(X, y),
                   callbacks=callbacks_list)

In [ ]:
t= (np.log1p(T)).reshape(-1, 96 * 96)
Tin= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1); 
Tin= Tin.reshape(-1, 96, 96)
Tin= np.expand_dims(Tin, axis= -1)
Tin.shape

In [ ]:
pX= model.predict(Xin[:300])
pT= model.predict(Tin[:300])

In [ ]:
pX= np.squeeze(pX, axis=-1)
pT= np.squeeze(pT, axis=-1)

In [ ]:
level= pX.flatten()
y_f= y[:300].flatten()
dice= []
thresholds= np.arange(0.1, level.max(), 0.01)
for threshold in thresholds:
    p_test= np.zeros((level.shape))

    p_test[level > threshold]= 1
    p_test= p_test.astype(np.float32)
    y_true= y_f.astype(np.float32)

    dice.append(dice_coef1(y_true, p_test.squeeze()))
plt.plot(thresholds, dice)
max_dice= np.array(dice).max()
best_th= thresholds[np.argmax(np.array(dice))]
print('max dice is ' + str(max_dice) + 'at threshold ' + str(best_th))

In [ ]:
p_finalX= np.zeros((pX.shape))
p_finalX[pX > best_th]= 1

p_finalT= np.zeros((pT.shape))
p_finalT[pT > best_th]= 1

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3(X[i], p_finalX[i]  , y[i])

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3(T[i], p_finalT[i]  , p_finalT[i])

## Level1

In [ ]:
D1E10_comp1= np.load(mid_path + 'predD1E10.npy'); D1E10T_comp1= np.load(mid_path + 'predD1E10T.npy')
D2E50_comp1= np.load(mid_path + 'predD2E50.npy'); D2E50T_comp1= np.load(mid_path + 'predD2E50T.npy')
X= np.load(mid_path + 'X96.npy'); T= np.load(mid_path + 'T96.npy')
y= np.load(mid_path + 'y96.npy')

In [ ]:
x= X.reshape(-1, 96 * 96); t= T.reshape(-1, 96 * 96)
orgX= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
orgT= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1); 

x= (np.log1p(X)).reshape(-1, 96 * 96); t= (np.log1p(T)).reshape(-1, 96 * 96)
log1pX= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
log1pT= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1); 

x= D1E10_comp1.reshape(-1, 96 * 96); t= D1E10T_comp1.reshape(-1, 96 * 96)
D1E10_comp1X= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
D1E10_comp1T= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)


x= (X - D1E10_comp1).reshape(-1, 96 * 96); t= (T - D1E10T_comp1).reshape(-1, 96 * 96)
D1E10_comp2X= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
D1E10_comp2T= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)

x= D2E50_comp1.reshape(-1, 96 * 96); 
D2E50_comp1X= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
D2E50_comp1T= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)

x= (X - D2E50_comp1).reshape(-1, 96 * 96); 
D2E50_comp2X= (x- np.expand_dims(x.mean(axis=1), axis=1))/np.expand_dims(x.std(axis=1), axis=1); 
D2E50_comp2T= (t- np.expand_dims(t.mean(axis=1), axis=1))/np.expand_dims(t.std(axis=1), axis=1)

In [ ]:
level1X= []
level1T= []
a= ['X_seg', 'log1pX_seg', 'D1E10_comp1_seg', 'D1E10_comp2_seg', 'D2E50_comp1_seg', 'D2E50_comp2_seg'] 
b= [orgX, log1pX, D1E10_comp1X, D1E10_comp2X, D2E50_comp1X, D2E50_comp2X]
c= [orgT, log1pT, D1E10_comp1T, D1E10_comp2T, D2E50_comp1T, D2E50_comp2T]

for m, data, dataT in zip(a, b, c):
    print(m)
    data= data.reshape(-1, 96, 96); data= np.expand_dims(data, axis= -1)
    dataT= dataT.reshape(-1, 96, 96); dataT= np.expand_dims(dataT, axis= -1)
    
    model = UNet((96, 96, 1), start_ch=16,depth=4,batchnorm=True)
    model.compile(optimizer = 'adadelta', loss = dice_coef_loss, metrics=[dice_coef])
    model.load_weights(models_path + m + '_weights_ep05_best.hdf5')
    #level1X.append(model.predict(data).flatten())
    level1T.append(model.predict(dataT).flatten())
    
#level1X= np.array(level1X).T 
level1T= np.array(level1T).T
#np.save(mid_path + 'level1X', level1X); 
np.save(mid_path + 'level1T', level1T)

In [ ]:
level1= np.load(mid_path + 'level1X.npy'); level1T= np.load(mid_path + 'level1T.npy')
#y_f= np.load(mid_path + 'y96.npy').flatten();

In [ ]:
best1= []
level= level1
for col in range(level.shape[1]):
    dice= []
    thresholds= np.arange(0.1, level[:, col].max(), 0.01)
    for threshold in thresholds:
        p_test= np.zeros((level.shape[0], 1))

        p_test[level[:, col] > threshold]= 1
        p_test= p_test.astype(np.float32)
        y_true= y_f.astype(np.float32)

        dice.append(dice_coef1(y_true, p_test.squeeze()))
    plt.plot(thresholds, dice)
    max_dice= np.array(dice).max()
    best_th= thresholds[np.argmax(np.array(dice))]
    best1.append(best_th)
    print('max dice is ' + str(max_dice) + ' at threshold ' + str(best_th))

In [ ]:
p_finalT= np.zeros((level1T.shape[0], 1))
p_finalT[level1T[:, 3] > 0.15]= 1

# X 0.39
# log 0.49
#     0.5
# 0.31

## Level2

In [ ]:
level1X= np.load(mid_path + 'level1X.npy'); 
level1T= np.load(mid_path + 'level1T.npy'); 
y_f= np.load(mid_path + 'y96.npy').flatten(); 
X96= np.load(mid_path + 'X96.npy')
T96= np.load(mid_path + 'T96.npy')
d_f= np.load(mid_path + 'd_f96.npy'); dt_f= np.load(mid_path + 'dt_f96.npy'); 

In [ ]:
d_f= d_f.reshape(-1, 1)
dt_f= dt_f.reshape(-1, 1)

scaler = MinMaxScaler()
limit= d_f.shape[0]
d_N= scaler.fit_transform(np.vstack((d_f, dt_f))).astype(np.float32)
d_f= d_N[:limit, :]
dt_f= d_N[limit:, :]

In [ ]:
dt_f.shape, level1T.shape

In [ ]:
X_level2= level1X
T_level2= level1T

In [ ]:
X_level2= np.hstack((X_level2, d_f)) 
T_level2= np.hstack((T_level2, dt_f)) 
X_level2.shape

In [ ]:
level2= []
level2T= []

print('BR')
level2_BRmodel= BayesianRidge()
level2_BRmodel.fit(X_level2, y_f)
level2_pBR= level2_BRmodel.predict(X_level2)


print(level2_pBR.max(), level2_BRmodel.coef_)
level2.append(level2_pBR)

level2T.append(level2_BRmodel.predict(T_level2))


print('LGB')
lgb_params= {'feature_fraction': 0.7,'metric': 'rmse', 'nthread':12, 'min_data_in_leaf': 2**7, 
              'bagging_fraction': 0.7, 'learning_rate': 0.05, 'objective': 'rmse',
              'bagging_seed': 2**5, 'num_leaves': 2**11,'bagging_freq':1,'verbosity': 50}

level2_LGBmodel = lgb.train(lgb_params, lgb.Dataset(X_level2, label=y_f), 50)
p= level2_LGBmodel.predict(X_level2)
weight= y_f-p + 1

print('LGB_error')
level2_LGBmodel1 = lgb.train(lgb_params, lgb.Dataset(X_level2, label=y_f, weight=weight), 300)
p1= level2_LGBmodel1.predict(X_level2)
error1= y_f-p1

pT1= level2_LGBmodel1.predict(T_level2)

print('LGB_error1')
level2_LGBmodel2 = lgb.train(lgb_params, lgb.Dataset(X_level2, label=error1), 300)
p2= level2_LGBmodel2.predict(X_level2)

pT2= level2_LGBmodel2.predict(T_level2)

level2.append(p1 + p2)
level2T.append(pT1 + pT2)
#ax = lgb.plot_importance(level2_LGBmodel, figsize=(12, 30));plt.show()

level2= np.array(level2).T 
level2T= np.array(level2T).T 

level2.max(axis=0)    

In [ ]:
ax = lgb.plot_importance(level2_LGBmodel, figsize=(12, 30));plt.show()

In [ ]:
level2_LGBmodel1.save_model('level2_LGBmodel1.txt')
level2_LGBmodel2.save_model('level2_LGBmodel2.txt')

In [ ]:
np.save(mid_path + 'level2X', level2); np.save(mid_path + 'level2T', level2T)

## Final

In [ ]:
level2= np.load(mid_path + 'level2X.npy'); level2T= np.load(mid_path + 'level2T.npy')
y_f= np.load(mid_path + 'y96.npy').flatten();

In [ ]:
level= level2
for col in range(level.shape[1]):
    dice= []
    thresholds= np.arange(0.1, level[:, col].max(), 0.01)
    for threshold in thresholds:
        p_test= np.zeros((level.shape[0], 1))

        p_test[level[:, col] > threshold]= 1
        p_test= p_test.astype(np.float32)
        y_true= y_f.astype(np.float32)

        dice.append(dice_coef1(y_true, p_test.squeeze()))
    plt.plot(thresholds, dice)
    max_dice= np.array(dice).max()
    best_th= thresholds[np.argmax(np.array(dice))]
    print('max dice is ' + str(max_dice) + ' at threshold ' + str(best_th))

In [ ]:
p_final= np.zeros((level2.shape[0], 1))
p_final[level2[:, 0] > best_th]= 1

In [ ]:
p_finalT= np.zeros((level2T.shape[0], 1))
p_finalT[level2T[:, 0] > best_th]= 1

### QC

In [ ]:
imp= p_final.reshape((-1, 96, 96))
y_c= y_f.reshape((-1, 96, 96))

In [ ]:
for i in np.arange(0, 300, 5):
    print(i)
    show_im3(X96[i], y_c[i], imp[i])

In [ ]:
impT= p_finalT[:96 * 96 * 300].reshape((-1, 96, 96))

In [ ]:
for i in np.arange(0, 300, 2):
    print(i)
    show_im2(T96[i], impT[i])

# Results

In [ ]:
names= np.load(mid_path + 'names.npy'); names0= np.load(mid_path + 'names0.npy')

In [ ]:
pred_final= p_finalT.reshape((-1, 96, 96))

In [ ]:
rle_dict= {}
for im, name in zip(pred_final, names):
    im= np.squeeze(im)
    im= cv2.resize(im, (101, 101))
    rle_dict[name]= rle(im)

rle_dict0= {}
for name0 in names0:
    rle_dict0[name0]= rle(np.zeros((101, 101)))

rle_dict.update(rle_dict0)

sub = pd.DataFrame.from_dict(rle_dict, orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('True_submissionCNN_4.csv')